In [ ]:
"""
# For Google Colab training

!pip install pytorch_lightning pedalboard

from google.colab import drive
drive.mount('/content/drive')

from drive.MyDrive.apr.src.datasets import VoxCelebDataModule
from drive.MyDrive.apr.src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from drive.MyDrive.apr.src.models import (
    ResNet34, build_efficientnetv2,
    Var_ECAPA, MHA_LAS, ResNet34SE, ResNet20
)
from drive.MyDrive.apr.src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

seed_everything(42, workers=True)

!unzip "/content/drive/MyDrive/datasets/voxceleb1/subset3.zip" -d "/content/subset"
"""

write svc (MFCC?), write verification test dataset, create 6sec dataset

In [1]:
import warnings
warnings.filterwarnings(
    "ignore", ".*Trying to infer the `batch_size`.*"
)
warnings.filterwarnings(
    "ignore", ".*The dataloader,.*"
)

import torch
from torch import nn
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from src.datasets import VoxCelebDataModule
from src.losses import (
    AAMSoftmaxLoss, SubCenterAAMSoftmaxLoss, SoftmaxLoss
)
from src.models import (
    ResNet34, build_efficientnetv2,
    Var_ECAPA, MHA_LAS, ResNet34SE, ResNet20
)
from src.resnetse import ResNetSE, SEBasicBlock, ResNetSEV2

seed_everything(42, workers=True)

Global seed set to 42


42

In [2]:
num_classes = 100
batch_size = 64
n_mels = 80
from_memory = False
spec_augment = True
data_dir = "/media/gabriele/Seagate Expansion Drive/Datasets/VoxCeleb1/"
colab_data_dir = "/content/"
num_workers = 4

dm3 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=3,
    # num_workers=num_workers,
    # data_dir=data_dir
)
dm4 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=4,
    # num_workers=num_workers,
    # data_dir=data_dir
)
dm6 = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=6,
    # num_workers=num_workers,
    # data_dir=data_dir
)

dmn = VoxCelebDataModule(
    batch_size=batch_size,
    from_memory=from_memory,
    spec_augment=spec_augment,
    num_secs=None,
    # num_workers=num_workers,
    # data_dir=data_dir
)

# ResNet34

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=aams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Test (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
checkpoint_path = "checkpoints/resnet34/last.ckpt"
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512,
    scale=30,
    margin=0.2
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=512
)
model = ResNet34(num_classes=num_classes, loss_func=aams) \
        .load_from_checkpoint(
            checkpoint_path, 
            num_classes=num_classes, 
            loss_func=aams
        )
logger = TensorBoardLogger("tb_logs", name="resnet34")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    resume_from_checkpoint=checkpoint_path
)
trainer.test(
    model, 
    dm,
    ckpt_path=checkpoint_path
)

# ResNet34-SE

## Train (3 secs)

In [5]:
# 64 epochs with m 0.1 and s 15

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.2,
    scale=15,
    num_subcenters=3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr=0.0001,
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="3secs",
    version=1
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    precision=16,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Restoring states from the checkpoint path at checkpoints/resnet34se/3secs/epoch=6-step=6474.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Restored all states from the checkpoint file at checkpoints/resnet34se/3secs/epoch=6-step=6474.ckpt

   | Name           | Type                     | Params
-------------------------------------------------------------
0  | loss_func      | SubCenterAAMSoftmaxLoss  | 76.8 K
1  | train_acc      | Accuracy                 | 0     
2  | val_acc        | Accuracy                 | 0     
3  | train_top1_acc | Accuracy                 | 0     
4  | val_top1_acc   | Accuracy                 | 0     
5  | train_top5_acc | Accuracy                 | 0     
6  | val_top5_acc   | Accuracy                 | 0     
7  | train_f1       | F1Score                  | 0     
8  | val_f1         | F1Score     

Epoch 64:   1%|          | 8/934 [00:19<37:18,  2.42s/it, loss=0.00719, v_num=1, train_loss_step=0.00743, train_acc_step=1.000, train_f1_step=1.000, train_top1_acc_step=1.000, train_top1_f1_step=1.000, train_top5_acc_step=1.000, train_top5_f1_step=0.627, val_loss_step=0.874, val_acc_step=0.811, val_f1_step=0.849, val_top1_acc_step=0.811, val_top1_f1_step=0.849, val_top5_acc_step=0.865, val_top5_f1_step=0.779, val_loss_epoch=1.870, val_acc_epoch=0.619, val_f1_epoch=0.585, val_top1_acc_epoch=0.619, val_top1_f1_epoch=0.585, val_top5_acc_epoch=0.800, val_top5_f1_epoch=0.305, train_loss_epoch=0.00457, train_acc_epoch=0.999, train_f1_epoch=0.999, train_top1_acc_epoch=0.999, train_top1_f1_epoch=0.999, train_top5_acc_epoch=1.000, train_top5_f1_epoch=0.370]    

## Train (4 secs)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
logs_dir = "/content/drive/MyDrive/tb_logs"

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes,
    n_mels=n_mels,
    loss_func=softmax,
    optimizer="Adam",
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet34se",
    sub_dir="4secs"
)
trainer = Trainer(
    deterministic=True,
    gpus=1,
    gradient_clip_val=1,
    precision=16,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)

## Train (6 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet34se/6secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet34SE(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    lr_scheduler=None
)
logger = TensorBoardLogger("tb_logs", name="resnet34se")
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model=model, 
    datamodule=dm6,
    ckpt_path="checkpoints/resnet34se/last.ckpt"
)

# EfficientNetV2

## Train (3 secs)

In [ ]:
checkpoint_dir = "/content/drive/MyDrive/checkpoints/"
logs_dir = "/content/drive/MyDrive/tb_logs"

checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir + "efficientnetv2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=softmax,
    optimizer="Adam",
    # lr=0.001,
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger(
    logs_dir, 
    name="efficientnetv2",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    precision=16,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path=checkpoint_dir + "efficientnetv2/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/efficientnetv2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
model = build_efficientnetv2(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="RMSprop",
    #lr=0.001,
    #lr_scheduler="ReducePlateau",
    #lr_scheduler_interval="epoch"
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="efficientnetv2",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm4)

# MHA_LAS

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/MHA_LAS/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.2
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = MHA_LAS(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR"
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="mha_las",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm4)

# VAR_ECAPA

## Train (3 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = Var_ECAPA(
    num_classes=num_classes,
    embeddings_dim=256, 
    loss_func=aams
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="var_ecapa",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/var_ecapa/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_min_dcf",
    mode="min",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = Var_ECAPA(
    num_classes=num_classes,
    embeddings_dim=256, 
    loss_func=aams
)
logger = TensorBoardLogger(
    "tb_logs", 
    name="var_ecapa",
    sub_dir="4secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm4)

# ResNetSE

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetse/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [16, 32, 64, 128]
model = ResNetSE(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetse",
    sub_dir="3secs"
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
    # resume_from_checkpoint="checkpoints/historical_events/event/epoch=6-step=25654.ckpt"
)
trainer.fit(model, dm3)

# ResNetSEV2

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetsev2/3secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=30,
    margin=0.3
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [32, 64, 128, 256]
model = ResNetSEV2(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetsev2",
    sub_dir="3secs",
    version=0
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm3,
    ckpt_path="checkpoints/resnetsev2/3secs/last.ckpt"
)

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnetsev2/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
num_filters = [32, 64, 128, 256]
model = ResNetSEV2(
    SEBasicBlock,
    [3, 4, 6, 3],
    num_filters,
    nOut=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
)

logger = TensorBoardLogger(
    "tb_logs", 
    name="resnetsev2",
    sub_dir="4secs",
)
trainer = Trainer(
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnetsev2/3secs/last.ckpt"
)

# ResNet20

## Train (4 secs)

In [ ]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/resnet20/4secs/",
    every_n_epochs=1,
    save_top_k=2,
    monitor="val_f1",
    mode="max",
    save_last=True
)
aams = AAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    scale=15,
    margin=0.1
)
subaams = SubCenterAAMSoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256,
    margin=0.1,
    scale=15
)
softmax = SoftmaxLoss(
    num_classes=num_classes,
    embeddings_dim=256
)
model = ResNet20(
    embeddings_dim=256,
    num_classes=num_classes, 
    loss_func=subaams,
    optimizer="Adam",
    # lr_scheduler="CyclicLR",
    # lr_scheduler_interval="step"
)
logger = TensorBoardLogger(
    save_dir="tb_logs", 
    name="resnet20",
    sub_dir="4secs"
)
trainer = Trainer(
    deterministic=True,
    gpus=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0
)
trainer.fit(
    model, 
    dm4,
    # ckpt_path="checkpoints/resnet34se/3secs/last.ckpt"
)